In [59]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [17]:
df = pd.read_csv('data/merged_question.csv')

In [18]:
q = df['question']

In [19]:
f = open("data/all_questions.txt", "w") 
for row in q.values:
    f.write(str(row))
    f.write(str('\n'))

In [23]:
# load ascii text and covert to lowercase
filename = "data/all_questions.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list)

Total Characters:  629836
Total Vocab:  94
Total Patterns:  629736
Epoch 1/1
629736/629736 [==============================] - 2740s 4ms/step - loss: 2.8601

Epoch 00001: loss improved from inf to 2.86012, saving model to weights-improvement-01-2.8601.hdf5


In [24]:

# load the network weights
filename = "weights-improvement-01-2.8601.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:

int_to_char = dict((i, c) for i, c in enumerate(chars))


In [77]:

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
print(start)
print(pattern)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(300):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
# 	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print( "\nDone.")

3533
[56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41, 1, 56, 51, 1, 56, 44, 41, 1, 56, 51, 54, 41]
Seed:
"  magazine was it first published in?
his excellency, the right honorable ramon hnatyshyn, p.c., q.c. "

Done.


In [73]:
mask = np.column_stack([q.str.contains(r"what is found directly below “hot air”", na=False) for col in q])
q.loc[mask.any(axis=1)]

3214    In the latest map of online communities, what ...
4592    In the latest map of online communities, what ...
Name: question, dtype: object

In [74]:
q.iloc[3214]

'In the latest map of online communities, what is found directly below “hot air”?'